In [ ]:
from sympy import *
from IPython.display import display, Math, Latex, clear_output, Image
import numpy as np
import cupy as cp
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from skimage.feature import peak_local_max
from scipy.ndimage import center_of_mass
from scipy.spatial import Delaunay
from collections import Counter
import os

In [ ]:
import sys
sys.path.append('D:/GitHubWSU/pfc-blocks/python/pfc-models/ln-vacancy')

import importlib

import PFC2D_Vacancy as PFC2D_Vacancy
importlib.reload(PFC2D_Vacancy)

In [ ]:
outputRoot = "./out/prospectus/phase/"

In [ ]:
def Plot(sim, Plot_t=True, Plot_f=True, Show=True, file=None, zrange=None):
  if zrange is not None:
     zmin=zrange[0]
     zmax=zrange[1]
  else:
     zmin=sim.phi.get().min()
     zmax=sim.phi.get().max()
  fig = go.Figure(data=go.Heatmap(x=np.arange(sim.nx)*sim.dx,
                                  y=np.arange(sim.ny)*sim.dy,
                                  z=sim.phi.get(),
                                  zmin=zmin,
                                  zmax=zmax,
                                  colorscale='gray',
                                  showscale=False))

  # decide pixels per cell (1 means 1 figure pixel per heatmap cell)
  px_per_cell = 2

  # exact figure pixel size to match heatmap grid
  fig_width = int(sim.nx * px_per_cell)
  fig_height = int(sim.ny * px_per_cell)

  fig.update_layout(
      width=fig_width,
      height=fig_height,
      autosize=False,
      margin=dict(l=0, r=0, t=0, b=0),
      plot_bgcolor='white',
      # place colorbar inside plot or hide it to avoid extra space
      # if you want a colorbar keep it small and place it on top of the image
      # e.g. fig.data[0].colorbar.update({'len':1, 'thickness':8, 'x':0.995})
  )

  annotations = []
  if Plot_t:
    annotations.append(
        dict(
          text=f"t = {sim.t:.2f}",
          showarrow=False,
          xref="paper", yref="paper",
          x=0.05, y=0.9,
          xanchor="left", yanchor="top",
          font=dict(color="cyan", size=16, family="Arial"),
        )
    )
  if Plot_f:
    annotations.append(
        dict(
          text=f"f = {sim.f:.8f}",
          showarrow=False,
          xref="paper", yref="paper",
          x=0.95, y=0.1,
          xanchor="right", yanchor="bottom",
          font=dict(color="yellow", size=16, family="Arial"),
        )
    )
  fig.update_layout(annotations=annotations)

  fig.update_xaxes(
      showgrid=False, showticklabels=False,
      # force axis to use full horizontal domain
      domain=[0.0, 1.0],
      range=[0, sim.nx * sim.dx]   # match the heatmap coordinates
  )

  fig.update_yaxes(
      showgrid=False, showticklabels=False,
      domain=[0.0, 1.0],
      range=[0, sim.ny * sim.dy],  # match the heatmap coordinates
      scaleanchor='x',             # lock aspect ratio so a unit in x equals unit in y
      scaleratio=1
  )

  # optionally hide the colorbar to remove its margin impact
  if hasattr(fig.data[0], 'colorbar'):
      fig.data[0].colorbar.update({'thickness': 8, 'len': 1, 'x': 0.995})  # place thin bar on right
      # or remove it: fig.data[0].colorbar = dict() or fig.data[0].showscale = False

  if Show:
    fig.show()

  if file is not None:
    fig.write_image(file)

## 1. Standard PFC

### 1.1 Triangle Ground State

In [ ]:
beta_val = 0.25 # 1.0
epsilon_val = -0.2 #-2.82 -0.1
g_val = 0 #-1
phi_s = -0.20 # triangle
# phi_s = -0.0 # stripe
# phi_s = 0.23 # honeycomb
# phi_s = -.2 # liquid
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 0
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 40
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 2.5e-1 #0.0025
sim.parms.dt = 1e-2
sim.parms.seed = 5
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
sim.SetGeometry(288, 288, 40, scalefactor = 1/q_eff, forceUnitCellBoundary=True)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

sim.InitFieldCrystal(A = np.sign(sim.parms.phi0) * sim.parms.phi0/3.1, noisy=False, scalefactor = 1/q_eff)
# sim.InitFieldFlat(noisy=True)
# sim.phi = cp.sin(sim.x)
# sim.phi[288-20:288+20,288:288+40] = phi_l
# sim.phi[0:64,:] = phi_l
# sim.phi[-64:,:] = phi_l
# sim.phi[:,288-72:288+72] = sim.phi[288-72:288+72,:].T
sim.t = 0
sim.CalcEnergyDensity()

frame = 0
Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
frame += 1

# Total steps and checkpoints
sim.SetDT(1e-1)
total_steps = int(1e3)
energy_calcs = 10
frames = 1# 100

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Create frame
    if i % int(total_steps/frames) == int(total_steps/frames)-1:
      Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
      frame += 1

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim, False, False, Show=True, file=outputRoot + f"fig-std-triangle.png")
phi_std = sim.phi.get()

### 1.2 Stripe Ground State

In [ ]:
beta_val = 0.25 # 1.0
epsilon_val = -0.2 #-2.82 -0.1
g_val = 0 #-1
# phi_s = -0.20 # triangle
phi_s = -0.0 # stripe
# phi_s = 0.23 # honeycomb
# phi_s = -.2 # liquid
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 0
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 40
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 2.5e-1 #0.0025
sim.parms.dt = 1e-2
sim.parms.seed = 5
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
sim.SetGeometry(288, 288, 40, scalefactor = 1/q_eff, forceUnitCellBoundary=True)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

# sim.InitFieldCrystal(A = np.sign(sim.parms.phi0) * sim.parms.phi0/3.1, noisy=False, scalefactor = 1/q_eff)
# sim.InitFieldFlat(noisy=True)
sim.phi = cp.sin(sim.x)/2
# sim.phi[288-20:288+20,288:288+40] = phi_l
# sim.phi[0:64,:] = phi_l
# sim.phi[-64:,:] = phi_l
# sim.phi[:,288-72:288+72] = sim.phi[288-72:288+72,:].T
sim.t = 0
sim.CalcEnergyDensity()

frame = 0
Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
frame += 1

# Total steps and checkpoints
sim.SetDT(1e-1)
total_steps = int(1e3)
energy_calcs = 10
frames = 1# 100

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Create frame
    if i % int(total_steps/frames) == int(total_steps/frames)-1:
      Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
      frame += 1

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim, False, False, Show=True, file=outputRoot + f"fig-std-stripe.png")
phi_std = sim.phi.get()

### 1.3 Honeycomb Ground State

In [ ]:
beta_val = 0.25 # 1.0
epsilon_val = -0.2 #-2.82 -0.1
g_val = 0 #-1
# phi_s = -0.20 # triangle
# phi_s = -0.0 # stripe
phi_s = 0.20 # honeycomb
# phi_s = -.2 # liquid
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 0
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 40
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 2.5e-1 #0.0025
sim.parms.dt = 1e-2
sim.parms.seed = 5
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
sim.SetGeometry(288, 288, 40, scalefactor = 1/q_eff, forceUnitCellBoundary=True)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

sim.InitFieldCrystal(A = -np.sign(sim.parms.phi0) * sim.parms.phi0/3.1, noisy=False, scalefactor = 1/q_eff)
# sim.InitFieldFlat(noisy=True)
# sim.phi = cp.sin(sim.x)/2
# sim.phi[288-20:288+20,288:288+40] = phi_l
# sim.phi[0:64,:] = phi_l
# sim.phi[-64:,:] = phi_l
# sim.phi[:,288-72:288+72] = sim.phi[288-72:288+72,:].T
sim.t = 0
sim.CalcEnergyDensity()

frame = 0
Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
frame += 1

# Total steps and checkpoints
sim.SetDT(1e-1)
total_steps = int(1e3)
energy_calcs = 10
frames = 1# 100

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Create frame
    if i % int(total_steps/frames) == int(total_steps/frames)-1:
      Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
      frame += 1

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim, False, False, Show=True, file=outputRoot + f"fig-std-honeycomb.png")
phi_std = sim.phi.get()

## 2. Log PFC

### 1.1 Triangle Ground State

In [ ]:
beta_val = 0.289 # 1.0
epsilon_val = 0.769 #-2.82 -0.1
g_val = -2.5 #-1
phi_s = 0.813 # triangle
# phi_s = 1.0 # stripe
# phi_s = 1.18 # honeycomb
# phi_s = -.2 # liquid
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 1
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 40
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 2.5e-1 #0.0025
sim.parms.dt = 1e-2
sim.parms.seed = 5
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
sim.SetGeometry(288, 288, 40, scalefactor = 1/q_eff, forceUnitCellBoundary=True)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

sim.InitFieldCrystal(A = np.sign(sim.parms.phi0) * sim.parms.phi0/3.1, noisy=False, scalefactor = 1/q_eff)
# sim.InitFieldFlat(noisy=True)
# sim.phi = cp.sin(sim.x)
# sim.phi[288-20:288+20,288:288+40] = phi_l
# sim.phi[0:64,:] = phi_l
# sim.phi[-64:,:] = phi_l
# sim.phi[:,288-72:288+72] = sim.phi[288-72:288+72,:].T
sim.t = 0
sim.CalcEnergyDensity()

frame = 0
Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
frame += 1

# Total steps and checkpoints
sim.SetDT(1e-1)
total_steps = int(1e3)
energy_calcs = 10
frames = 1# 100

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Create frame
    if i % int(total_steps/frames) == int(total_steps/frames)-1:
      Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
      frame += 1

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim, False, False, Show=True, file=outputRoot + f"fig-log-triangle.png")
phi_std = sim.phi.get()

### 1.2 Stripe Ground State

In [ ]:
beta_val = 0.289 # 1.0
epsilon_val = 0.769 #-2.82 -0.1
g_val = -2.5 #-1
# phi_s = 0.813 # triangle
phi_s = 1.0 # stripe
# phi_s = 1.18 # honeycomb
# phi_s = -.2 # liquid
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 1
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 40
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 2.5e-1 #0.0025
sim.parms.dt = 1e-2
sim.parms.seed = 5
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
sim.SetGeometry(288, 288, 40, scalefactor = 1/q_eff, forceUnitCellBoundary=True)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

# sim.InitFieldCrystal(A = np.sign(sim.parms.phi0) * sim.parms.phi0/3.1, noisy=False, scalefactor = 1/q_eff)
# sim.InitFieldFlat(noisy=True)
sim.phi = cp.sin(sim.x)/2 + sim.parms.phi0
# sim.phi[288-20:288+20,288:288+40] = phi_l
# sim.phi[0:64,:] = phi_l
# sim.phi[-64:,:] = phi_l
# sim.phi[:,288-72:288+72] = sim.phi[288-72:288+72,:].T
sim.t = 0
sim.CalcEnergyDensity()

frame = 0
Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
frame += 1

# Total steps and checkpoints
sim.SetDT(1e-1)
total_steps = int(1e3)
energy_calcs = 10
frames = 1# 100

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Create frame
    if i % int(total_steps/frames) == int(total_steps/frames)-1:
      Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
      frame += 1

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim, False, False, Show=True, file=outputRoot + f"fig-log-stripe.png")
phi_std = sim.phi.get()

### 1.3 Honeycomb Ground State

In [ ]:
beta_val = 0.289 # 1.0
epsilon_val = 0.769 #-2.82 -0.1
g_val = -2.5 #-1
# phi_s = 0.813 # triangle
# phi_s = 1.0 # stripe
phi_s = 1.18 # honeycomb
# phi_s = -.2 # liquid
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 1
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 40
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 2.5e-1 #0.0025
sim.parms.dt = 1e-2
sim.parms.seed = 5
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
sim.SetGeometry(288, 288, 40, scalefactor = 1/q_eff, forceUnitCellBoundary=True)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

sim.InitFieldCrystal(A = -np.sign(sim.parms.phi0) * sim.parms.phi0/3.1, noisy=False, scalefactor = 1/q_eff)
sim.phi += -sim.phi.min()
sim.phi *= sim.parms.phi0/sim.phi.mean()
# sim.InitFieldFlat(noisy=True)
# sim.phi = cp.sin(sim.x)/2
# sim.phi[288-20:288+20,288:288+40] = phi_l
# sim.phi[0:64,:] = phi_l
# sim.phi[-64:,:] = phi_l
# sim.phi[:,288-72:288+72] = sim.phi[288-72:288+72,:].T
sim.t = 0
sim.CalcEnergyDensity()

frame = 0
Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
frame += 1

# Total steps and checkpoints
sim.SetDT(1e-1)
total_steps = int(1e3)
energy_calcs = 10
frames = 1# 100

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Create frame
    if i % int(total_steps/frames) == int(total_steps/frames)-1:
      Plot(sim)#, Show=False, file=outputRoot + f"liquid_{frame:04d}.png", zrange=(-1.0, 0.00))
      frame += 1

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

Plot(sim, False, False, Show=True, file=outputRoot + f"fig-log-honeycomb.png")
phi_std = sim.phi.get()